In [1]:
import pandas as pd
import numpy as np
import math
from math import ceil, floor
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure

In [ ]:
#convert nuclei to mnist type data

class SliceGenerator():
    def __init__(self, image_path, image_size):
        self.image_path = image_path
        self.image_size = image_size

    def slice_image(self):
        ex_nuc = pd.read_csv(self.image_path)
        ex_nuc = ex_nuc.astype(float)

        minx = np.min(ex_nuc["x"])
        miny = np.min(ex_nuc["y"])

        image_base = np.zeros((self.image_size, self.image_size))
        x_padding = (self.image_size - (np.max(ex_nuc["x"]) - np.min(ex_nuc["x"])))/2
        y_padding = (self.image_size - (np.max(ex_nuc["y"]) - np.min(ex_nuc["y"])))/2
        x_left = ceil(x_padding)
        y_left = ceil(y_padding)

        ex_nuc["new_y"] = ex_nuc["y"] - miny + y_left
        ex_nuc["new_x"] = ex_nuc["x"] - minx + x_left

        image_set = []
        for i in set(ex_nuc["z"].values): #slice
            image_base = np.zeros((self.image_size, self.image_size))
            for row in range(0, len(ex_nuc["z"])-1): #index in ex_nuc
                if ex_nuc["z"].values[row] == i: #just focussing on slice by slice
                    if int(ex_nuc["new_x"].values[row]) >= self.image_size or int(ex_nuc["new_y"].values[row]) >= self.image_size:
                        print(int(ex_nuc["new_x"].values[row])-1,int(ex_nuc["new_y"].values[row])-1)
                    image_base[int(ex_nuc["new_x"].values[row])-1,int(ex_nuc["new_y"].values[row])-1] = 1 #0 index
                else:
                    pass
            image_set.append(image_base)
        
        return image_set

    def plot_image(self):
        image_set = self.slice_image()
        fig = plt.figure(figsize=(20, 20))
        for i in range(0, len(image_set)):
            img = image_set[i]
            plt.subplot(6,6,i+1)
            plt.axis("off")
            plt.imshow(img, cmap="gray")
    
    def unravel_slices(self):
        image_set = self.slice_image()
        unravelled = []
        for array in image_set:    
            unravelled.extend(np.ravel(array))
        
        return unravelled
    
    def plot_3D(self):

        image_set = self.slice_image()
        ex = np.stack(image_set)

        # Use marching cubes to obtain the surface mesh of these ellipsoids
        verts, faces, normals, values = measure.marching_cubes_lewiner(ex, 0)

        # Display resulting triangular mesh using Matplotlib. This can also be done
        # with mayavi (see skimage.measure.marching_cubes_lewiner docstring).
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')

        # Fancy indexing: `verts[faces]` to generate a collection of triangles
        mesh = Poly3DCollection(verts[faces])
        mesh.set_edgecolor('k')
        ax.add_collection3d(mesh)


        ax.set_xlabel("x-axis: a = 6 per ellipsoid")
        ax.set_ylabel("y-axis: b = 10")
        ax.set_zlabel("z-axis: c = 16")

        ax.set_xlim(0, self.image_size)  # a = 6 (times two for 2nd ellipsoid)
        ax.set_ylim(0, self.image_size)  # b = 10
        ax.set_zlim(0, self.image_size)  # c = 16


        plt.tight_layout()
        base = os.path.basename(os.path.normpath(self.image_path))
        plt.savefig(base+".png")
        plt.close(fig)